# Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

# Funciones

### Función para generar nuevos boletos aleatorios

In [ ]:
def randomgen():

    numero = rd.sample(range(1,50),7)
    complementario = rd.sample(numero,1)
    numero.remove(complementario[0])
    reintegro = rd.sample(range(10),1)
    
    numero.sort()

    combo = {}
    combo['numero'] = numero
    combo['reintegro'] = reintegro

    return combo

### Función para generar nuevos boletos teniendo en cuenta la distribución de la muestra histórica

In [ ]:
def frequencygen(df, exp=3):
    #obtengo las números sacados en orden cronológico
    tiradas = []
    for row in df.index:
        for n in df.loc[row,'1':'6']:
            tiradas.append(n)
    tiradas = pd.Series(tiradas)

    #obtengo la frecuencia de muestreo de cada número
    freqs = tiradas.value_counts()
    #si algún número no ha salido en la muestra, lo añado
    for i in range(1,50):
        if not i in freqs.index:
            freqs[i] = 0
    #los ordeno por el índice
    freqs = freqs.sort_index()

    #obtengo la desviación de la frecuencia de cada número respecto a la frecuencia media
    desviacion = freqs-freqs.mean()
    #invierto la frecuencia para que esté en el signo correcto
    i_desviacion = -desviacion
    #obtengo la probabilidad histórica de cada número, y genero una lista con tantas repeticiones por número como probabilidades tenga
    probs = (i_desviacion/len(tiradas))/2
    probs *= np.pi
    probs = np.tan(probs)
    adding = np.round((np.abs(probs)**(1/exp))*(probs/np.abs(probs))*100,0).fillna(0)

    #esta es la lista
    ruleta = []
    for i in range(1, 50):
        ns = int(100 + adding[i])
        for n in range(ns):
            ruleta.append(i)

    #obtengo una combinación eligiendo números aleatorios de la lista
    numeros = []
    for i in range(6):
        numeros.append(rd.sample(ruleta, 1)[0])
        indexes = np.where(np.array(ruleta)==numeros[-1])[0].tolist()
        ruleta = ruleta[:indexes[0]] + ruleta[(indexes[-1]+1):]
    numeros.sort()

    #obtengo las frecuencias históricas de los reintegros y añado los que no hayan salido en el histórico
    freqs = df.loc[:,'R'].value_counts()
    for i in range(10):
        if not i in freqs.index:
            freqs[i] = 0
    freqs = freqs.sort_index()
    #obtengo la desviación de la frecuencia de cada número con respecto a la frecuencia media
    desviacion = freqs-freqs.mean()
    #lo invierto para que esté bien el signo
    i_desviacion = -desviacion

    #obtengo la probabilidad de cada número, y genero una lista con tantas repeticiones del número como probabilidades tenga
    probs = (i_desviacion/len(df.loc[:,'R']))/2
    probs *= np.pi
    probs = np.tan(probs)
    adding = np.round((np.abs(probs)**(1/exp))*(probs/np.abs(probs))*100,0).fillna(0)

    #esta es la lista
    ruleta = []
    for i in range(10):
        ns = int(100 + adding[i])
        for n in range(ns):
            ruleta.append(i)

    #escojo un número aleatorio de la lista generada
    reintegro = rd.sample(ruleta, 1)

    #genero la respuesta de la función
    boleto = {
        'numero':numeros,
        'reintegro':reintegro
    }

    return boleto

### Función de comprobación del premio dado un boleto y la combinación ganadora

In [ ]:
def check_primitiva(boleto, sorteo):

    n_match = 0
    for n in boleto['numero']:
        if n in sorteo['numero']:
            n_match+=1

    r_match = boleto['reintegro'] == sorteo['reintegro']

    premio = '0'

    if n_match == 6 and r_match:
        premio = 'E'
    elif n_match == 6:
        premio = '1'
    elif n_match == 5 and sorteo['complementario'][0] in boleto['numero']:
        premio = '2'
    elif n_match == 5:
        premio = '3'
    elif n_match == 4:
        premio = '4'
    elif n_match == 3:
        premio == '5'
    elif r_match:
        premio = 'R'

    return premio

# Simulación

### Obtención de datos históricos de la primitiva y homogeneización

In [ ]:
path13_22='https://docs.google.com/spreadsheets/d/e/2PACX-1vTov1BuA0nkVGTS48arpPFkc9cG7B40Xi3BfY6iqcWTrMwCBg5b50-WwvnvaR6mxvFHbDBtYFKg5IsJ/pub?gid=1&single=true&output=csv'
path85_12='https://docs.google.com/spreadsheets/d/e/2PACX-1vTov1BuA0nkVGTS48arpPFkc9cG7B40Xi3BfY6iqcWTrMwCBg5b50-WwvnvaR6mxvFHbDBtYFKg5IsJ/pub?gid=0&single=true&output=csv'

file2 = pd.read_csv(path13_22)
file2 = file2.set_index('FECHA')
file2 = file2.iloc[:,:-1]
file2.columns = ['1','2','3','4','5','6','C','R']

file1 = pd.read_csv(path85_12)
file1 = file1.set_index('FECHA')
file1 = file1.iloc[:,:-1]
file1.columns = ['1','2','3','4','5','6','C','R']
file1 = file1.iloc[:-310]

file = pd.concat([file2, file1])

In [ ]:
file.head()

### Simulación Aleatorio vs. Javi

In [ ]:
#variables optimizables
long_muestra = 1000
exponente = 3

resultados_aleatorio = []
resultados_javi = []

n_sims = 1000
for i in range(n_sims):
    inicio = np.random.randint(0,(file.shape[0] - long_muestra + 1))
    fin = inicio + long_muestra

    muestra = file[inicio:fin]
    boleto_random = randomgen()
    boleto_javi = frequencygen(muestra, exponente)
    ganador = {
        'numero':[int(i) for i in file.iloc[fin][:6]],
        'reintegro':[int(file.iloc[fin][7])],
        'complementario':[int(file.iloc[fin][6])]
    }

    resultados_aleatorio.append(check_primitiva(boleto_random, ganador))
    resultados_javi.append(check_primitiva(boleto_javi, ganador))

print('Boleto aleatorio')
print(pd.Series(resultados_aleatorio).value_counts())
print('-----------------')
print('Boleto de Javi')
print(pd.Series(resultados_javi).value_counts())

### Cuánto dinero se ha ganado

In [ ]:
#pendiente
categorias = ['0','R','5','4','3','2','1','E']
premios = {
    '0':0,
    'R':1,
    '5':8,
    '4':30,
    '3':1_050,
    '2':50_000,
    '1':1_300_000,
    'E':24_500_000
}
premios = pd.Series(premios)